In [59]:
import pandas as pd
import os
# from bs4 import BeautifulSoup
# from splinter import Browser
# import pprint

In [75]:
def stdName(df,col_to_split):
    
    #Separate out players first and last name for joining later
    df[["FirstNm","Last"]] = df[col_to_split].str.split(" ",n=1,expand=True)
    df['FirstNm']=df['FirstNm'].str.upper()
    df['Last']=df['Last'].str.upper()

    #Get rid of , and . which may be inconsistent based on sample
    df['Last']=df['Last'].str.replace('.','')
    df['Last']=df['Last'].str.replace(',','')
    df['FirstNm']=df['FirstNm'].str.replace('.','')
    df['FirstNm']=df['FirstNm'].str.replace(',','')

    #Add preceeding , to common suffixes for parsing out
    df['Last']=df['Last'].str.replace('JR',',JR')
    df['Last']=df['Last'].str.replace('II',',II')
    df['Last']=df['Last'].str.replace('III',',III')
    df['Last']=df['Last'].str.replace('IV',',IV')

    #parse out the suffix from the last name
    df[['LastNm','Suffix']]=df['Last'].str.split(",",n=1,expand=True)

    return df

In [60]:
def get_csv(filename):
    path = os.path.abspath('')
    filepath = f"{path}/FPTS/{filename}.csv"
    return filepath

In [91]:
#Function to replace null values with an average factored by a penalty
def fill_null_floats(df,col_headers,penalty=0):
    df = df.fillna("NoRecord")
    for col in col_headers:
        mean_df = df[df[col]!='NoRecord']
        average = mean_df[col].astype(float).mean()
        print(average)
        df[col]=df[col].replace(to_replace='NoRecord',value = (average-penalty))
    return df

In [61]:
files = ['2008Fpts','2009Fpts','2010Fpts','2011Fpts','2012Fpts','2013Fpts','2014Fpts','2015Fpts','2016Fpts','2017Fpts','2018Fpts','2019Fpts']

In [94]:
fpt_df =[]
for file in files:
    path = (get_csv(file))
    df = pd.read_csv(path)
    fpt_df.append(df)

In [111]:
Fantasy_df = pd.concat(fpt_df)
sum_fant = Fantasy_df.groupby('Player').sum()
sum_fant.reset_index(inplace = True)
sum_fant_df=stdName(sum_fant,'Player')
sum_fant_df

,Player,Year,FPTS,FirstNm,Last,LastNm,Suffix
0,A Allison,2008,21.0,A,ALLISON,ALLISON,None
1,A Becht,2008,9.9,A,BECHT,BECHT,None
2,A Lelie,2008,42.7,A,LELIE,LELIE,None
3,A.J. Brown,2019,217.1,AJ,BROWN,BROWN,None
4,A.J. Green,16116,1866.0,AJ,GREEN,GREEN,None
...,...,...,...,...,...,...,...
710,Will Fuller V,8070,475.7,WILL,FULLER V,FULLER V,None
711,Willie Snead IV,10085,619.6,WILLIE,"SNEAD ,IV",SNEAD,IV
712,Yamon Figurs,6027,16.1,YAMON,FIGURS,FIGURS,None
713,Zach Pascal,4037,202.1,ZACH,PASCAL,PASCAL,None


,Year,Player,FPTS,FirstNm,Last,LastNm,Suffix
0,2008,Andre Johnson,320.5,ANDRE,JOHNSON,JOHNSON,None
1,2008,Larry Fitzgerald,311.1,LARRY,FITZGERALD,FITZGERALD,None
2,2008,Calvin Johnson,281.0,CALVIN,JOHNSON,JOHNSON,None
3,2008,Roddy White,266.6,RODDY,WHITE,WHITE,None
4,2008,Greg Jennings,265.2,GREG,JENNINGS,JENNINGS,None
...,...,...,...,...,...,...,...
208,2019,Dwayne Harris,1.7,DWAYNE,HARRIS,HARRIS,None
209,2019,Jaleel Scott,1.6,JALEEL,SCOTT,SCOTT,None
210,2019,Krishawn Hogan,1.4,KRISHAWN,HOGAN,HOGAN,None
211,2019,Cyril Grayson,1.3,CYRIL,GRAYSON,GRAYSON,None


In [76]:
college_df = pd.read_csv("allcollegestat.csv")
college_df=stdName(college_df,'Player')
college_df.drop(columns = ['careergames','Unnamed: 0'], inplace = True)
#fill missing college stats with 0
college_df.fillna(0,inplace = True)

In [77]:


college_df

,careerRec,careerRecYds,careerYPC,careerRecTds,careerRushAtt,careerRushYds,careerYPA,careerRushTds,careerPlays,careerTotalYds,...,dr_3YPA,dr_3RushTds,dr_3Plays,dr_3TotalYds,dr_3YPT,dr_3TotalTds,FirstNm,Last,LastNm,Suffix
0,161.0,1743.0,10.8,9.0,117.0,594.0,5.1,8.0,278,2337,...,4.0,1.0,76,585,7.7,2,DERRICK,WILLIAMS,WILLIAMS,0
1,101.0,1910.0,18.9,15.0,11.0,94.0,8.5,0.0,112,2004,...,2.0,0.0,25,412,16.5,2,MIKE,WALLACE,WALLACE,0
2,132.0,1931.0,14.6,16.0,19.0,63.0,3.3,0.0,151,1994,...,0.0,0.0,23,290,12.6,4,TIQUAN,UNDERWOOD,UNDERWOOD,0
3,138.0,1752.0,12.7,17.0,0.0,0.0,0.0,0.0,138,1752,...,0.0,0.0,29,272,9.4,2,PATRICK,TURNER,TURNER,0
4,259.0,3231.0,12.5,22.0,45.0,395.0,8.8,3.0,304,3626,...,11.3,0.0,56,665,11.9,2,MIKE,THOMAS,THOMAS,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,136.0,1976.0,14.5,16.0,40.0,375.0,9.4,7.0,176,2351,...,10.4,1.0,28,375,13.4,3,RODNEY,ADAMS,ADAMS,0
311,119.0,1832.0,15.4,12.0,10.0,112.0,11.2,1.0,129,1944,...,0.0,0.0,0,0,0.0,0,MARQUEZ,VALDES-SCANTLING,VALDES-SCANTLING,0
312,136.0,1512.0,11.1,13.0,3.0,11.0,3.7,0.0,139,1523,...,5.5,0.0,19,204,10.7,0,TREY,QUINN,QUINN,0
313,146.0,2037.0,14.0,17.0,14.0,53.0,3.8,1.0,160,2090,...,5.5,1.0,39,515,13.2,5,KJ,OSBORN,OSBORN,0


In [73]:
combine_df = pd.read_csv('combine.csv')
combine_df

,Unnamed: 0,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,...,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm
0,0,1,2009,Derrick Williams,WR,22.0,Penn State,194,4.47,33.0,...,NaN,NaN,NaN,Detroit Lions,3,82,2009,71,DERRICK,WILLIAMS
1,1,2,2009,Mike Wallace,WR,22.0,Mississippi,199,4.33,40.0,...,129.0,6.90,4.27,Pittsburgh Steelers,3,84,2009,72,MIKE,WALLACE
2,2,3,2009,Tiquan Underwood,WR,22.0,Rutgers,184,4.31,41.5,...,129.0,6.62,4.11,Jacksonville Jaguars,7,253,2009,73,TIQUAN,UNDERWOOD
3,3,4,2009,Patrick Turner,WR,21.0,USC,223,4.59,NaN,...,NaN,NaN,NaN,Miami Dolphins,3,87,2009,77,PATRICK,TURNER
4,4,5,2009,Mike Thomas,WR,21.0,Arizona,195,4.30,40.5,...,126.0,6.65,4.28,Jacksonville Jaguars,4,107,2009,68,MIKE,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,46,47,2020,Tyrie Cleveland,WR,NaN,Florida,209,4.46,39.5,...,126.0,NaN,NaN,Denver Broncos,7,252,2020,74,TYRIE,CLEVELAND
332,47,48,2020,Chase Claypool,WR,21.0,Notre Dame,238,4.42,40.5,...,126.0,NaN,NaN,Pittsburgh Steelers,2,49,2020,76,CHASE,CLAYPOOL
333,48,49,2020,Quintez Cephus,WR,NaN,Wisconsin,202,4.73,38.5,...,124.0,7.20,4.33,Detroit Lions,5,166,2020,73,QUINTEZ,CEPHUS
334,53,53,2020,Lynn Bowden,WR,22.0,Kentucky,204,NaN,NaN,...,NaN,NaN,NaN,Las Vegas Raiders,3,80,2020,71,LYNN,BOWDEN


In [85]:
collcomb = college_df.merge(combine_df, on = ['FirstNm','LastNm'], how = 'left')

collcomb.drop(index = [292,295], inplace = True)

In [104]:
draft2020 = collcomb[collcomb['year']==2020]
simulation_data = collcomb[collcomb['year']!=2020]
simulation_data[simulation_data['year']==2019]

,careerRec,careerRecYds,careerYPC,careerRecTds,careerRushAtt,careerRushYds,careerYPA,careerRushTds,careerPlays,careerTotalYds,...,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in)
237,79.0,1605.0,20.3,11.0,0.0,0.0,0.0,0.0,79,1605,...,40.5,11.0,135.0,7.00,4.15,New York Giants,5.0,171.0,2019.0,73.0
238,148.0,2076.0,14.0,16.0,25.0,154.0,6.2,7.0,173,2230,...,39.0,15.0,122.0,7.03,4.14,San Francisco 49ers,2.0,36.0,2019.0,71.0
239,69.0,1015.0,14.7,13.0,4.0,46.0,11.5,0.0,73,1061,...,30.5,13.0,124.0,7.22,4.28,Chicago Bears,4.0,126.0,2019.0,73.0
240,186.0,2133.0,11.5,15.0,0.0,0.0,0.0,0.0,186,2133,...,35.0,7.0,116.0,6.80,4.19,Oakland Raiders,5.0,149.0,2019.0,70.0
241,119.0,1710.0,14.4,14.0,5.0,11.0,2.2,0.0,124,1721,...,36.5,12.0,122.0,NaN,4.29,Minnesota Vikings,7.0,239.0,2019.0,73.0
242,67.0,1228.0,18.3,14.0,0.0,0.0,0.0,0.0,67,1228,...,40.5,27.0,134.0,7.38,4.50,Seattle Seahawks,2.0,64.0,2019.0,75.0
243,75.0,1251.0,16.7,19.0,0.0,0.0,0.0,0.0,75,1251,...,37.5,18.0,125.0,7.01,4.15,Washington Redskins,3.0,76.0,2019.0,72.0
244,125.0,2019.0,16.2,11.0,1.0,13.0,13.0,0.0,126,2032,...,38.0,14.0,124.0,6.88,4.16,Minnesota Vikings,7.0,247.0,2019.0,72.0
245,275.0,3463.0,12.6,24.0,8.0,27.0,3.4,0.0,283,3490,...,30.0,14.0,117.0,7.28,4.23,Arizona Cardinals,6.0,174.0,2019.0,73.0
246,135.0,2235.0,16.6,23.0,4.0,26.0,6.5,0.0,139,2261,...,33.5,15.0,123.0,7.09,4.45,Pittsburgh Steelers,3.0,66.0,2019.0,70.0


In [120]:
#list of columns that need to be floats and adjusted to null values
col_replace = ['40YD','Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle']

#replace null values and convert measurables to floats using function defined above
simul_data = fill_null_floats(simulation_data,col_replace)

4.4775187969924755
35.79535864978903
14.7981220657277
121.65517241379311
6.903863636363638
4.220706521739131


In [121]:
simul_data

,careerRec,careerRecYds,careerYPC,careerRecTds,careerRushAtt,careerRushYds,careerYPA,careerRushTds,careerPlays,careerTotalYds,...,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in)
0,161.0,1743.0,10.8,9.0,117.0,594.0,5.1,8.0,278,2337,...,33.000000,15.000000,121.655172,6.903864,4.220707,Detroit Lions,3,82,2009,71
1,101.0,1910.0,18.9,15.0,11.0,94.0,8.5,0.0,112,2004,...,40.000000,14.000000,129.000000,6.900000,4.270000,Pittsburgh Steelers,3,84,2009,72
2,132.0,1931.0,14.6,16.0,19.0,63.0,3.3,0.0,151,1994,...,41.500000,7.000000,129.000000,6.620000,4.110000,Jacksonville Jaguars,7,253,2009,73
3,138.0,1752.0,12.7,17.0,0.0,0.0,0.0,0.0,138,1752,...,35.795359,14.798122,121.655172,6.903864,4.220707,Miami Dolphins,3,87,2009,77
4,259.0,3231.0,12.5,22.0,45.0,395.0,8.8,3.0,304,3626,...,40.500000,14.798122,126.000000,6.650000,4.280000,Jacksonville Jaguars,4,107,2009,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,214.0,3177.0,14.8,34.0,2.0,7.0,3.5,0.0,216,3184,...,41.000000,14.000000,131.000000,6.840000,4.080000,Washington Redskins,1,22,2016,74
311,109.0,1672.0,15.3,12.0,4.0,3.0,0.8,0.0,113,1675,...,38.500000,11.000000,124.000000,6.600000,4.220000,Green Bay Packers,5,163,2016,73
312,136.0,1976.0,14.5,16.0,40.0,375.0,9.4,7.0,176,2351,...,29.500000,8.000000,125.000000,6.980000,4.280000,Minnesota Vikings,5,170,2017,73
313,119.0,1832.0,15.4,12.0,10.0,112.0,11.2,1.0,129,1944,...,30.500000,15.000000,124.000000,6.903864,4.220707,Green Bay Packers,5,174,2018,76


In [122]:
collfant = simul_data.merge(sum_fant_df, on = ['FirstNm','LastNm'], how = 'left')

In [123]:
collfant['FPTS'].fillna("NoPoints", inplace = True)
left_out = collfant[collfant['FPTS']=="NoPoints"]
uni = left_out['Player_x'].unique()

In [124]:
model_df = collfant[collfant['FPTS']!="NoPoints"]
model_df.to_csv('model_df.csv')

In [125]:
model_df

,careerRec,careerRecYds,careerYPC,careerRecTds,careerRushAtt,careerRushYds,careerYPA,careerRushTds,careerPlays,careerTotalYds,...,Team,Round,pickNum,year,height(in),Player,Year,FPTS,Last_y,Suffix_y
0,161.0,1743.0,10.8,9.0,117.0,594.0,5.1,8.0,278,2337,...,Detroit Lions,3,82,2009,71,Derrick Williams,4019.0,15.6,WILLIAMS,None
1,101.0,1910.0,18.9,15.0,11.0,94.0,8.5,0.0,112,2004,...,Pittsburgh Steelers,3,84,2009,72,Mike Wallace,18117.0,1705.3,WALLACE,None
2,132.0,1931.0,14.6,16.0,19.0,63.0,3.3,0.0,151,1994,...,Jacksonville Jaguars,7,253,2009,73,Tiquan Underwood,8046.0,199.7,UNDERWOOD,None
3,138.0,1752.0,12.7,17.0,0.0,0.0,0.0,0.0,138,1752,...,Miami Dolphins,3,87,2009,77,Patrick Turner,4021.0,28.2,TURNER,None
4,259.0,3231.0,12.5,22.0,45.0,395.0,8.8,3.0,304,3626,...,Jacksonville Jaguars,4,107,2009,68,Mike Thomas,14094.0,438.8,THOMAS,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,147.0,2114.0,14.4,17.0,7.0,64.0,9.1,0.0,154,2178,...,Tennessee Titans,3,77,2010,73,Damian Williams,14077.0,333.4,WILLIAMS,None
276,214.0,3177.0,14.8,34.0,2.0,7.0,3.5,0.0,216,3184,...,Washington Redskins,1,22,2016,74,Josh Doctson,6051.0,237.6,DOCTSON,None
277,109.0,1672.0,15.3,12.0,4.0,3.0,0.8,0.0,113,1675,...,Green Bay Packers,5,163,2016,73,Trevor Davis,6052.0,51.1,DAVIS,None
279,119.0,1832.0,15.4,12.0,10.0,112.0,11.2,1.0,129,1944,...,Green Bay Packers,5,174,2018,76,Marquez Valdes-Scantling,4037.0,195.1,VALDES-SCANTLING,None


In [137]:
#Drop Avg columns to combine seasons, they will be re-calculated later
comb_df.drop(columns = ['Avg','Avg.1','Avg.2'], inplace=True)

#Get rid of extra header rows
comb_df = comb_df[comb_df['Rk'] != 'Rk']

#Convert Statistic Columns to integers
comb_df[['Rec', 'Yds', 'TD', 'Att', 'Yds.1', 'TD.1', 'Plays', 'Yds.2', 'TD.2']] = comb_df[['Rec', 'Yds', 'TD', 'Att','Yds.1', 'TD.1', 'Plays', 'Yds.2', 'TD.2']].astype(int)

#Strip the * symbol from player which indicates bowl stats are included
comb_df['Player']=comb_df['Player'].map(lambda x: x.rstrip('*'))

#Combine stats of players by grouping by Player and School
college_career_df = comb_df.groupby(['Player']).sum()

#Reset Index
college_career_df = college_career_df.reset_index()

#Standardize Names of Players to match with other dataset
college_career_df[['FirstNm','Last']]=college_career_df['Player'].str.split(" ",n=1,expand=True)
college_career_df['FirstNm']=college_career_df['FirstNm'].str.upper()
college_career_df['Last']=college_career_df['Last'].str.upper()
college_career_df['FirstNm']=college_career_df['FirstNm'].str.replace('.','')
college_career_df['FirstNm']=college_career_df['FirstNm'].str.replace(',','')
college_career_df['Last']=college_career_df['Last'].str.replace('.','')
college_career_df['Last']=college_career_df['Last'].str.replace(',','')



In [138]:
college_career_df

,Player,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,FirstNm,Last
0,A.J. Barker,30,577,7,0,0,0,30,577,7,AJ,BARKER
1,A.J. Bennett,60,511,4,6,-2,0,66,509,4,AJ,BENNETT
2,A.J. Brown,189,2984,19,0,0,0,189,2984,19,AJ,BROWN
3,A.J. Butler,32,213,0,50,177,1,82,390,1,AJ,BUTLER
4,A.J. Coney,43,551,3,0,0,0,43,551,3,AJ,CONEY
...,...,...,...,...,...,...,...,...,...,...,...,...
3689,Zane Fakes,98,871,7,7,48,0,105,919,7,ZANE,FAKES
3690,Zane Pope,47,525,1,1,0,0,48,525,1,ZANE,POPE
3691,Zavier Scott,33,228,1,34,210,0,67,438,1,ZAVIER,SCOTT
3692,Zay Jones,399,4279,23,5,18,0,404,4297,23,ZAY,JONES


In [64]:
# college_career_df= college_career_df.merge(years, how = 'inner', on = 'Player')
# college_career_df=college_career_df.rename(columns = {'School_y':'Years Played','School_x':'School'})
splitsuffix=college_career_df

In [72]:
splitsuffix['Last']=splitsuffix['Last'].str.replace('JR',',JR')
splitsuffix['Last']=splitsuffix['Last'].str.replace('II',',II')
splitsuffix['Last']=splitsuffix['Last'].str.replace('III',',III')
splitsuffix['Last']=splitsuffix['Last'].str.replace('IV',',IV')

In [73]:
splitsuffix[['LastNm','Suffix']]=college_career_df['Last'].str.split(",",n=1,expand=True)


In [186]:
players = splitsuffix['Player'].unique()
len(players)

3694

In [139]:
splitsuffix[splitsuffix['Player']=='Zay Jones']

,Player,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,FirstNm,Last,LastNm,Suffix
3692,Zay Jones,399,4279,23,5,18,0,404,4297,23,ZAY,JONES,JONES,None


In [146]:
athlete=[]
yearsplayed=[]
for p in players:
    try:
        years = comb_df[comb_df['Player']==p]['year'].unique().astype(int)
        athlete.append(p)
        yearsplayed.append(len(years))
    except:
        print("years not defined")

In [154]:
df['years'].unique()
over4 = ['James Washington','Mike Davis','Anthony Johnson','Chris Givens','Chris Harper','Deon Butler','Kyle Williams','Marcus Green','Matt Miller','Mike Williams']

array([1, 3, 4, 2, 5, 6], dtype=int64)

In [216]:
df=pd.DataFrame({'player':athlete,'years':yearsplayed})
df4 = df[df['years']==4]
played1year = df[df['years']==1]
yp1 = played1year['player'].unique()
len(yp1)

1768

In [198]:
played2year = df[df['years']==2]
yp2 = played2year['player'].unique()
yp2

array(['A.J. Jenkins', 'A.J. Love', 'A.J. Richardson', ..., 'Zack Moss',
       'Zack Sledge', 'Zane Fakes'], dtype=object)

In [202]:
played3year = df[df['years']==3]
yp3 = played3year['player'].unique()
yp3

array(['A.J. Brown', 'A.J. Green', 'A.J. Guyton', 'Aaron Bradley',
       'Aaron Burbridge', 'Aaron Dobson', 'Aaron Fuller',
       'Adam Humphries', 'Ajalen Holley', 'Al-Damion Riles', 'Alan Cross',
       'Albert Okwuegbunam', 'Albert Young', 'Aldrick Robinson',
       'Aleva Hifo', 'Alex Torres', 'Allen Hurns', 'Alshon Jeffery',
       'Amara Darboh', 'Amari Cooper', 'Andre Davis',
       'Andrew Bodenheimer', 'Andy Isabella', 'Antavian Edison',
       'Anthony McClung', 'Anthony Russo', 'Antonio Brown',
       'Antwan Dixon', 'Antwon Bailey', 'Arceto Clark', 'Aregeros Turner',
       'Arkeel Newsome', 'Armand Robinson', 'Armando Allen',
       'Arrelious Benn', 'Artavis Scott', 'Austin Franklin',
       'Austin Seferian-Jenkins', 'Baron Batch', 'Bernard Reedy',
       'Bert Reed', 'Bobby Ratliff', 'Bobby Swigert', 'Bradley Marquez',
       'Branden Ledbetter', 'Brandin Cooks', 'Brandon Allen',
       'Brandon Barden', 'Brandon Burks', 'Brandon Gibson',
       'Brandon Jackson', 'Br

In [152]:
for i in over4:
    years = comb_df[comb_df['Player']==i]['year'].unique().astype(int)

    print(years)
    print(len(years))

[2010 2011 2014 2015 2016 2017]
6
[2007 2010 2011 2012 2013 2014]
6
[2005 2006 2007 2017 2018]
5
[2007 2008 2009 2010 2011]
5
[2010 2011 2012 2013 2014]
5
[2005 2006 2007 2008 2014]
5
[2007 2009 2017 2018 2019]
5
[2009 2015 2016 2017 2018]
5
[2005 2011 2012 2013 2014]
5
[2006 2007 2009 2014 2016]
5


In [110]:
# college_career_df[['LastNm','Suffix']]=college_career_df['Last'].str.split(" ",n=1,expand=True)
renfrow1.columns

Index(['Rk', 'Player', 'School', 'Conf', 'G', 'Rec', 'Yds', 'TD', 'Att',
       'Yds.1', 'TD.1', 'Plays', 'Yds.2', 'TD.2', 'year'],
      dtype='object')

In [166]:
grouped = comb_df['Player'].unique()
grouped

array(['Matt Miller', 'Billy Pittman', 'Mario Urrutia', ...,
       'Kennedy McKoy', 'Alex Fontenot', 'SaRodorick Thompson'],
      dtype=object)

In [205]:
playersmultschools=[]
for i in grouped:
    schools = comb_df[comb_df['Player']==i]['School'].unique()
    if len(schools) >1:
#         print(f'{i} played at {schools}')
        playersmultschools.append(i)
    else:
        pass

In [207]:
len(playersmultschools)
combine_df

,Unnamed: 0,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,...,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm
0,0,1,2009,Derrick Williams,WR,22.0,Penn State,194,4.47,33.0,...,NaN,NaN,NaN,Detroit Lions,3,82,2009,71,DERRICK,WILLIAMS
1,1,2,2009,Mike Wallace,WR,22.0,Mississippi,199,4.33,40.0,...,129.0,6.90,4.27,Pittsburgh Steelers,3,84,2009,72,MIKE,WALLACE
2,2,3,2009,Tiquan Underwood,WR,22.0,Rutgers,184,4.31,41.5,...,129.0,6.62,4.11,Jacksonville Jaguars,7,253,2009,73,TIQUAN,UNDERWOOD
3,3,4,2009,Patrick Turner,WR,21.0,USC,223,4.59,NaN,...,NaN,NaN,NaN,Miami Dolphins,3,87,2009,77,PATRICK,TURNER
4,4,5,2009,Mike Thomas,WR,21.0,Arizona,195,4.30,40.5,...,126.0,6.65,4.28,Jacksonville Jaguars,4,107,2009,68,MIKE,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,46,47,2020,Tyrie Cleveland,WR,NaN,Florida,209,4.46,39.5,...,126.0,NaN,NaN,Denver Broncos,7,252,2020,74,TYRIE,CLEVELAND
332,47,48,2020,Chase Claypool,WR,21.0,Notre Dame,238,4.42,40.5,...,126.0,NaN,NaN,Pittsburgh Steelers,2,49,2020,76,CHASE,CLAYPOOL
333,48,49,2020,Quintez Cephus,WR,NaN,Wisconsin,202,4.73,38.5,...,124.0,7.20,4.33,Detroit Lions,5,166,2020,73,QUINTEZ,CEPHUS
334,53,53,2020,Lynn Bowden,WR,22.0,Kentucky,204,NaN,NaN,...,NaN,NaN,NaN,Las Vegas Raiders,3,80,2020,71,LYNN,BOWDEN


In [212]:
#Read in combine data of those that were drafted
combine_df = pd.read_csv('combine.csv')
combine_df.head(2)

#loop through each player with multiple schools and store their name if they were drafted
draftedmultiple=[]
for i in playersmultschools:
    df = combine_df[combine_df['Player']==i]
    count = df['Rk'].count()
    if count > 0:
        draftedmultiple.append(i)
    else:
        pass

#print list of drafted that went to multiple schools to decide on manual processing
print(draftedmultiple)

['Deon Butler', 'Mike Williams', 'Chris Givens', 'Anthony Miller', 'Jalen Saunders', 'Chris Harper', 'T.J. Graham', 'James Washington', 'Trevor Davis', 'Josh Doctson', 'Michael Thomas', 'Jalen Hurd', 'Van Jefferson', 'K.J. Osborn']


In [213]:
draftedmultiple

['Deon Butler',
 'Mike Williams',
 'Chris Givens',
 'Anthony Miller',
 'Jalen Saunders',
 'Chris Harper',
 'T.J. Graham',
 'James Washington',
 'Trevor Davis',
 'Josh Doctson',
 'Michael Thomas',
 'Jalen Hurd',
 'Van Jefferson',
 'K.J. Osborn']

In [219]:
#research into players that have 1 season of stats and were drafted
drafted1yearplayed = []
for i in yp1:
    df = combine_df[combine_df['Player']==i]
    count = df['Rk'].count()
    if count > 0:
        drafted1yearplayed.append(i)
    else:
        pass

#print list of drafted that went to multiple schools to decide on manual processing
print(len(drafted1yearplayed))

28


In [220]:
#research into players that have 2 season of stats and were drafted
drafted2yearplayed = []
for i in yp2:
    df = combine_df[combine_df['Player']==i]
    count = df['Rk'].count()
    if count > 0:
        drafted2yearplayed.append(i)
    else:
        pass

#print list of drafted that went to multiple schools to decide on manual processing
print(len(drafted2yearplayed))

105


In [217]:
df10 = comb_df[comb_df['Player']=='Riley Ridley']
df10

,Rk,Player,School,Conf,G,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,year
219,210,Riley Ridley,Georgia,SEC,14,43,559,9,0,0,0,43,559,9,2018


In [221]:
#research into players that have 2 season of stats and were drafted
drafted3yearplayed = []
for i in yp3:
    df = combine_df[combine_df['Player']==i]
    count = df['Rk'].count()
    if count > 0:
        drafted3yearplayed.append(i)
    else:
        pass

#print list of drafted that went to multiple schools to decide on manual processing
print(len(drafted3yearplayed))

114


In [184]:
givens = comb_df[comb_df['Player']=='K.J. Osborn']
givens

,Rk,Player,School,Conf,G,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,year
161,155,K.J. Osborn,Buffalo,MAC,12,35,493,4,4,22,1,39,515,5,2017
49,48,K.J. Osborn,Buffalo,MAC,13,53,892,7,5,16,0,58,908,7,2018
354,339,K.J. Osborn,Miami (FL),ACC,13,50,547,5,5,15,0,55,562,5,2019


In [1]:
def reccolumnsrename(df,prefix):
    df.rename(columns={'G':f'{prefix}games',
                        'Rec':f'{prefix}Rec',
                        'Yds':f'{prefix}RecYds',
                        'TD':f'{prefix}RecTds',
                        'Att':f'{prefix}RushAtt',
                        'Yds.1':f'{prefix}RushYds',
                        'TD.1':f'{prefix}RushTds',
                        'Plays':f'{prefix}Plays',
                        'Yds.2':f'{prefix}TotalYds',
                        'TD.2':f'{prefix}TotalTds'}, inplace = True)
    return df

In [2]:
def append0less3(df):
    df['dr_2games']=0,
    df['dr_2Rec']=0,
    df['dr_2RecYds']=0,
    df['dr_2YPC']=0,
    df['dr_2RecTds']=0,
    df['dr_2RushAtt']=0,
    df['dr_2RushYds']=0,
    df['dr_2YPA']=0,
    df['dr_2RushTds']=0,
    df['dr_2Plays']=0,
    df['dr_2TotalYds']=0,
    df['dr_2YPT']=0,
    df['dr_2TotalTds']=0,
    df['dr_3games']=0,
    df['dr_3Rec']=0,
    df['dr_3RecYds']=0,
    df['dr_3YPC']=0,
    df['dr_3RecTds']=0,
    df['dr_3RushAtt']=0,
    df['dr_3RushYds']=0,
    df['dr_3YPA']=0,
    df['dr_3RushTds']=0,
    df['dr_3Plays']=0,
    df['dr_3TotalYds']=0,
    df['dr_3YPT']=0,
    df['dr_3TotalTds']=0
    return df

In [ ]:
'dr_2games', 'dr_2Rec', 'dr_2RecYds',
       'dr_2YPC', 'dr_2RecTds', 'dr_2RushAtt', 'dr_2RushYds', 'dr_2YPA',
       'dr_2RushTds', 'dr_2Plays', 'dr_2TotalYds', 'dr_2YPT', 'dr_2TotalTds',
       'dr_3games', 'dr_3Rec', 'dr_3RecYds', 'dr_3YPC', 'dr_3RecTds',
       'dr_3RushAtt', 'dr_3RushYds', 'dr_3YPA', 'dr_3RushTds', 'dr_3Plays',
       'dr_3TotalYds', 'dr_3YPT', 'dr_3TotalTds'

In [117]:
renfrow1 = comb_df[(comb_df['Player']=='Hunter Renfrow') & (comb_df['year'] == years[0])]
renfrow2 = comb_df[(comb_df['Player']=='Hunter Renfrow') & (comb_df['year'] == years[1])]
renfrow3 = comb_df[(comb_df['Player']=='Hunter Renfrow') & (comb_df['year'] == years[2])]
renfrow4 = comb_df[(comb_df['Player']=='Hunter Renfrow') & (comb_df['year'] == years[3])]
renfrow2.rename(columns={'Rec':f'{years[1]}Rec',
                         'Yds':f'{years[1]}RecYds',
                         'TD':f'{years[1]}RecTds',
                         'Att':f'{years[1]}RushAtt',
                         'Yds.1':f'{years[1]}RushYds',
                         'TD.1':f'{years[1]}RushTds',
                         'Plays':f'{years[1]}Plays',
                         'Yds.2':f'{years[1]}TotalYds',
                         'TD.2':f'{years[1]}TotalTds'}, inplace = True)
renfrow1.rename(columns={'Rec':f'{years[0]}Rec',
                         'Yds':f'{years[0]}RecYds',
                         'TD':f'{years[0]}RecTds',
                         'Att':f'{years[0]}RushAtt',
                         'Yds.1':f'{years[0]}RushYds',
                         'TD.1':f'{years[0]}RushTds',
                         'Plays':f'{years[0]}Plays',
                         'Yds.2':f'{years[0]}TotalYds',
                         'TD.2':f'{years[0]}TotalTds'}, inplace = True)
renfrow2

C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Rk,Player,School,Conf,G,2016Rec,2016RecYds,2016RecTds,2016RushAtt,2016RushYds,2016RushTds,2016Plays,2016TotalYds,2016TotalTds,year
341,326,Hunter Renfrow,Clemson,ACC,11,44,495,6,0,0,0,44,495,6,2016


In [118]:
renf = renfrow1.merge(renfrow2,how='inner', on = 'Player')
renf

,Rk_x,Player,School_x,Conf_x,G_x,2015Rec,2015RecYds,2015RecTds,2015RushAtt,2015RushYds,...,2016Rec,2016RecYds,2016RecTds,2016RushAtt,2016RushYds,2016RushTds,2016Plays,2016TotalYds,2016TotalTds,year_y
0,109,Hunter Renfrow,Clemson,ACC,13,33,492,5,0,0,...,44,495,6,0,0,0,44,495,6,2016


In [48]:
# college_career_df.to_csv('collegestats.csv')
combine_df = pd.read_csv('collegestats.csv')
combine_df
# if college_career_df['Last'].split(" ").str[-1]
#     college_career_df['Last']=college_career_df['Last'].str.split()
final = combine_df.merge(college_career_df, how = 'inner',on = ['FirstNm','LastNm'])
final

,Unnamed: 0,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,...,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm
0,0,1,2009,Derrick Williams,WR,22.0,Penn State,194,4.470000,33.000000,...,121.942652,6.919082,4.226256,Detroit Lions,3,82,2009,71,DERRICK,WILLIAMS
1,1,2,2009,Mike Wallace,WR,22.0,Mississippi,199,4.330000,40.000000,...,129.000000,6.900000,4.270000,Pittsburgh Steelers,3,84,2009,72,MIKE,WALLACE
2,2,3,2009,Tiquan Underwood,WR,22.0,Rutgers,184,4.310000,41.500000,...,129.000000,6.620000,4.110000,Jacksonville Jaguars,7,253,2009,73,TIQUAN,UNDERWOOD
3,3,4,2009,Patrick Turner,WR,21.0,USC,223,4.590000,35.907018,...,121.942652,6.919082,4.226256,Miami Dolphins,3,87,2009,77,PATRICK,TURNER
4,4,5,2009,Mike Thomas,WR,21.0,Arizona,195,4.300000,40.500000,...,126.000000,6.650000,4.280000,Jacksonville Jaguars,4,107,2009,68,MIKE,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,46,47,2020,Tyrie Cleveland,WR,NoRecord,Florida,209,4.460000,39.500000,...,126.000000,6.919082,4.226256,Denver Broncos,7,252,2020,74,TYRIE,CLEVELAND
332,47,48,2020,Chase Claypool,WR,21,Notre Dame,238,4.420000,40.500000,...,126.000000,6.919082,4.226256,Pittsburgh Steelers,2,49,2020,76,CHASE,CLAYPOOL
333,48,49,2020,Quintez Cephus,WR,NoRecord,Wisconsin,202,4.730000,38.500000,...,124.000000,7.200000,4.330000,Detroit Lions,5,166,2020,73,QUINTEZ,CEPHUS
334,53,53,2020,Lynn Bowden,WR,22,Kentucky,204,4.475823,35.907018,...,121.942652,6.919082,4.226256,Las Vegas Raiders,3,80,2020,71,LYNN,BOWDEN


,Unnamed: 0,Rk,Year,Player_x,Pos,Age,School,Wt,40YD,Vertical,...,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,Last,Suffix
0,0,1,2009,Derrick Williams,WR,22.0,Penn State,194,4.470000,33.000000,...,1454,8,95,489,5,234,1943,13,WILLIAMS,None
1,1,2,2009,Mike Wallace,WR,22.0,Mississippi,199,4.330000,40.000000,...,1910,15,11,94,0,112,2004,15,WALLACE,None
2,2,3,2009,Tiquan Underwood,WR,22.0,Rutgers,184,4.310000,41.500000,...,1594,12,0,0,0,105,1594,12,UNDERWOOD,None
3,3,4,2009,Patrick Turner,WR,21.0,USC,223,4.590000,35.907018,...,1582,15,0,0,0,126,1582,15,TURNER,None
4,4,5,2009,Mike Thomas,WR,21.0,Arizona,195,4.300000,40.500000,...,3231,22,45,395,3,304,3626,25,THOMAS,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,46,47,2020,Tyrie Cleveland,WR,NoRecord,Florida,209,4.460000,39.500000,...,761,3,2,6,0,49,767,3,CLEVELAND,None
303,47,48,2020,Chase Claypool,WR,21,Notre Dame,238,4.420000,40.500000,...,2078,19,2,-3,0,147,2075,19,CLAYPOOL,None
304,48,49,2020,Quintez Cephus,WR,NoRecord,Wisconsin,202,4.730000,38.500000,...,1402,13,4,5,0,93,1407,13,CEPHUS,None
305,53,53,2020,Lynn Bowden,WR,22,Kentucky,204,4.475823,35.907018,...,1093,6,194,1493,13,291,2586,19,BOWDEN JR,JR


In [ ]:
final.rename(columns = {'Yds':'RecYds','TD':'RecTD','Att':'RushAtt','Yds.1':'RushYds','TD.1':'RushTD',
                       'Yds.2':'YFS','TD.2':'TotalTds',''})

In [51]:
final.to_csv("final.csv")

In [88]:
college_df
# ['School'].unique()

,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2
Player,,,,,,,,,
A.J. Barker,30,577,7,0,0,0,30,577,7
A.J. Bennett,60,511,4,6,-2,0,66,509,4
A.J. Brown,189,2984,19,0,0,0,189,2984,19
A.J. Butler,32,213,0,50,177,1,82,390,1
A.J. Coney,43,551,3,0,0,0,43,551,3
...,...,...,...,...,...,...,...,...,...
Zane Fakes,98,871,7,7,48,0,105,919,7
Zane Pope,47,525,1,1,0,0,48,525,1
Zavier Scott,33,228,1,34,210,0,67,438,1


In [80]:
players = comb_df['Player'].unique()

In [81]:
len(players)

2827

In [223]:
coldf=pd.read_csv('collegestats.csv')
messedup = coldf[coldf['Rec']<1]
messedup

,Unnamed: 0,level_0,index,Player,Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2,School,FirstNm,Last,LastNm,Suffix,Years Played
